# 🚀 MiniLLM 0.2B 模型训练 (A100 GPU)

本 Colab Notebook 在 A100 GPU 上从零训练一个 0.2B (200M 参数) 的 MiniLLM 模型。

**功能特点:**
- DeepSeek-V3.2 风格架构 (MLA + MoE + MTP)
- 支持预训练和 SFT 微调
- 针对 A100 80GB 优化的批量大小
- 支持混合精度训练 (bfloat16)

**运行环境要求:**
- Colab Pro+ 或自建 A100 GPU 环境
- Python 3.10+
- PyTorch 2.0+

## 1️⃣ 环境准备

In [ ]:
# 检查 GPU 类型
!nvidia-smi --query-gpu=name,memory.total --format=csv

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

In [ ]:
# 克隆 mini-llm 仓库
!git clone https://github.com/ai-clarify/mini-llm.git
%cd mini-llm

In [ ]:
# 安装依赖
!pip install -q torch==2.6.0 transformers==4.57.1 datasets==2.21.0 \
    tokenizers==0.22.0 safetensors==0.4.5 einops==0.8.1 \
    tensorboard==2.17.1 tqdm==4.66.5 pyyaml==6.0.2 \
    huggingface_hub==0.34.0 wandb==0.18.3 swanlab==0.6.8

# 验证安装
import transformers
print(f"Transformers version: {transformers.__version__}")

## 2️⃣ 模型配置 (0.2B 参数)

设计一个约 200M 参数的模型:
- `hidden_size=768`: 隐藏层维度
- `num_hidden_layers=12`: Transformer 层数
- `num_attention_heads=12`: 注意力头数
- `q_lora_rank=256`: Q LoRA 压缩维度
- `kv_lora_rank=128`: KV LoRA 压缩维度

**参数估算:**
- Embedding: vocab_size × hidden_size = 6400 × 768 ≈ 5M
- Attention: 12 层 × (MLA 参数) ≈ 80M
- FFN: 12 层 × (3 × hidden_size × intermediate) ≈ 100M
- 总计: ~200M 参数

In [ ]:
import sys
sys.path.append('.')

from model.model_minillm import MiniLLMConfig, MiniLLMForCausalLM

# 0.2B 模型配置
config_0_2b = MiniLLMConfig(
    vocab_size=6400,
    hidden_size=768,
    intermediate_size=2048,  # ~2.67x hidden_size
    num_hidden_layers=12,
    num_attention_heads=12,
    
    # MLA 配置
    q_lora_rank=256,
    kv_lora_rank=128,
    qk_nope_head_dim=64,
    qk_rope_head_dim=32,
    v_head_dim=64,
    
    # MTP (Multi-Token Prediction)
    num_nextn_predict_layers=1,
    mtp_loss_weight=0.1,
    
    # 训练相关
    max_position_embeddings=2048,
    rms_norm_eps=1e-6,
    dropout=0.0,
    attention_dropout=0.0,
    
    # 不使用 MoE (保持 Dense 模型)
    use_moe=False,
)

# 创建模型并计算参数量
model = MiniLLMForCausalLM(config_0_2b)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"\n📊 模型统计:")
print(f"总参数量: {total_params:,} ({total_params/1e6:.2f}M)")
print(f"可训练参数量: {trainable_params:,} ({trainable_params/1e6:.2f}M)")
print(f"\n📐 模型配置:")
print(f"  hidden_size: {config_0_2b.hidden_size}")
print(f"  num_hidden_layers: {config_0_2b.num_hidden_layers}")
print(f"  num_attention_heads: {config_0_2b.num_attention_heads}")
print(f"  intermediate_size: {config_0_2b.intermediate_size}")
print(f"  max_position_embeddings: {config_0_2b.max_position_embeddings}")

## 3️⃣ 数据准备

下载并准备预训练数据集。可选择:
1. 使用项目自带的示例数据
2. 从 HuggingFace 下载公开数据集
3. 使用自定义数据

In [ ]:
import os
import json
from datasets import load_dataset

# 创建数据目录
os.makedirs('data', exist_ok=True)

# 选项 1: 使用小型示例数据快速验证
USE_SAMPLE_DATA = True  # 设为 False 使用完整数据

if USE_SAMPLE_DATA:
    # 创建一个小型示例数据集用于验证流程
    sample_texts = [
        "人工智能正在改变世界，深度学习是其中最重要的技术之一。",
        "大语言模型可以理解和生成人类语言，它们在各种任务中表现出色。",
        "Transformer 架构是现代语言模型的基础，它使用自注意力机制来处理序列。",
        "预训练和微调是训练语言模型的两个主要阶段。",
        "多头注意力允许模型同时关注不同位置的信息。",
    ] * 1000  # 复制以创建足够的训练数据
    
    with open('data/pretrain_sample.jsonl', 'w', encoding='utf-8') as f:
        for text in sample_texts:
            f.write(json.dumps({'text': text}, ensure_ascii=False) + '\n')
    
    DATA_PATH = 'data/pretrain_sample.jsonl'
    print(f"✅ 已创建示例数据: {DATA_PATH}")
    print(f"   样本数量: {len(sample_texts)}")

else:
    # 选项 2: 从 HuggingFace 下载中文预训练数据
    print("正在下载预训练数据集...")
    
    # 使用 Wikipedia 中文或其他开源数据
    # 这里以 wikitext 为例，实际可替换为中文数据集
    dataset = load_dataset('wikitext', 'wikitext-2-raw-v1', split='train')
    
    with open('data/pretrain_full.jsonl', 'w', encoding='utf-8') as f:
        for item in dataset:
            if item['text'].strip():
                f.write(json.dumps({'text': item['text']}, ensure_ascii=False) + '\n')
    
    DATA_PATH = 'data/pretrain_full.jsonl'
    print(f"✅ 数据下载完成: {DATA_PATH}")

In [ ]:
# 查看数据格式
!head -3 {DATA_PATH}

## 4️⃣ 预训练

使用 `train_pretrain.py` 脚本进行预训练。

**A100 80GB 优化参数:**
- `batch_size=64`: 大批量提升吞吐量
- `accumulation_steps=4`: 等效 batch_size=256
- `max_seq_len=2048`: 完整上下文长度
- `dtype=bfloat16`: A100 原生支持的精度

In [ ]:
# 训练超参数
TRAINING_CONFIG = {
    # 模型参数 (0.2B)
    'hidden_size': 768,
    'num_hidden_layers': 12,
    
    # 训练参数
    'epochs': 3,
    'batch_size': 64,           # A100 可支持大批量
    'accumulation_steps': 4,    # 等效 batch_size=256
    'learning_rate': 3e-4,
    'max_seq_len': 2048,
    
    # 精度与优化
    'dtype': 'bfloat16',        # A100 原生支持
    'grad_clip': 1.0,
    
    # 日志与保存
    'log_interval': 50,
    'save_interval': 500,
    
    # 数据路径
    'data_path': DATA_PATH,
    'out_dir': 'out/pretrain_0_2b',
}

print("📋 训练配置:")
for k, v in TRAINING_CONFIG.items():
    print(f"  {k}: {v}")

In [ ]:
# 创建输出目录
import os
os.makedirs(TRAINING_CONFIG['out_dir'], exist_ok=True)

# 构建训练命令
train_cmd = f"""
python trainer/train_pretrain.py \
    --hidden_size {TRAINING_CONFIG['hidden_size']} \
    --num_hidden_layers {TRAINING_CONFIG['num_hidden_layers']} \
    --epochs {TRAINING_CONFIG['epochs']} \
    --batch_size {TRAINING_CONFIG['batch_size']} \
    --accumulation_steps {TRAINING_CONFIG['accumulation_steps']} \
    --learning_rate {TRAINING_CONFIG['learning_rate']} \
    --max_seq_len {TRAINING_CONFIG['max_seq_len']} \
    --dtype {TRAINING_CONFIG['dtype']} \
    --grad_clip {TRAINING_CONFIG['grad_clip']} \
    --log_interval {TRAINING_CONFIG['log_interval']} \
    --save_interval {TRAINING_CONFIG['save_interval']} \
    --data_path {TRAINING_CONFIG['data_path']} \
    --out_dir {TRAINING_CONFIG['out_dir']} \
    --tensorboard_dir {TRAINING_CONFIG['out_dir']}/tensorboard \
    --device cuda:0
"""

print("🚀 训练命令:")
print(train_cmd)

In [ ]:
# 开始预训练
# 注意: 完整训练可能需要几个小时到几天，取决于数据量
!{train_cmd}

## 5️⃣ 监控训练进度

使用 TensorBoard 可视化训练曲线。

In [ ]:
# 加载 TensorBoard
%load_ext tensorboard
%tensorboard --logdir out/pretrain_0_2b/tensorboard

## 6️⃣ SFT 微调 (可选)

预训练完成后，可以进行监督微调 (SFT) 来提升对话能力。

In [ ]:
# 准备 SFT 数据集
sft_samples = [
    {
        "instruction": "什么是人工智能？",
        "output": "人工智能(AI)是计算机科学的一个分支，旨在创建能够模拟人类智能的系统。"
    },
    {
        "instruction": "解释一下深度学习。",
        "output": "深度学习是机器学习的子领域，使用多层神经网络来学习数据的复杂表示。"
    },
    {
        "instruction": "Transformer 是什么？",
        "output": "Transformer 是一种神经网络架构，使用自注意力机制来处理序列数据，是现代语言模型的基础。"
    },
] * 500

with open('data/sft_sample.jsonl', 'w', encoding='utf-8') as f:
    for item in sft_samples:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print(f"✅ 已创建 SFT 数据: data/sft_sample.jsonl")
print(f"   样本数量: {len(sft_samples)}")

In [ ]:
# SFT 训练命令
sft_cmd = f"""
python trainer/train_full_sft.py \
    --hidden_size 768 \
    --num_hidden_layers 12 \
    --epochs 3 \
    --batch_size 32 \
    --accumulation_steps 4 \
    --learning_rate 2e-5 \
    --max_seq_len 2048 \
    --dtype bfloat16 \
    --data_path data/sft_sample.jsonl \
    --pretrained_path {TRAINING_CONFIG['out_dir']}/pretrain_768.pth \
    --out_dir out/sft_0_2b \
    --device cuda:0
"""

print("🎯 SFT 训练命令:")
print(sft_cmd)

In [ ]:
# 运行 SFT (取消注释以执行)
# !{sft_cmd}

## 7️⃣ 模型推理测试

In [ ]:
import torch
from transformers import AutoTokenizer
from model.model_minillm import MiniLLMConfig, MiniLLMForCausalLM

# 加载模型
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 使用预训练模型路径
MODEL_PATH = f"{TRAINING_CONFIG['out_dir']}/pretrain_768.pth"

# 重新创建模型配置
config = MiniLLMConfig(
    hidden_size=768,
    num_hidden_layers=12,
    num_attention_heads=12,
    q_lora_rank=256,
    kv_lora_rank=128,
)

model = MiniLLMForCausalLM(config)

# 加载权重
if os.path.exists(MODEL_PATH):
    state_dict = torch.load(MODEL_PATH, map_location=device)
    model.load_state_dict(state_dict)
    print(f"✅ 模型加载成功: {MODEL_PATH}")
else:
    print(f"⚠️ 模型文件不存在: {MODEL_PATH}")
    print("  请先运行预训练步骤")

model = model.to(device)
model.eval()

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained('./model/')

In [ ]:
def generate_text(prompt, max_new_tokens=100, temperature=0.7, top_p=0.9):
    """生成文本"""
    inputs = tokenizer(prompt, return_tensors='pt').to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    generated = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated

# 测试生成
test_prompts = [
    "人工智能",
    "深度学习是",
    "Transformer 模型",
]

print("📝 生成测试:")
print("=" * 50)
for prompt in test_prompts:
    try:
        result = generate_text(prompt, max_new_tokens=50)
        print(f"Prompt: {prompt}")
        print(f"Output: {result}")
        print("-" * 50)
    except Exception as e:
        print(f"生成失败: {e}")

## 8️⃣ 保存与导出

In [ ]:
# 保存模型到 HuggingFace 格式 (可选)
EXPORT_DIR = 'out/minillm_0_2b_hf'
os.makedirs(EXPORT_DIR, exist_ok=True)

# 保存配置
config.save_pretrained(EXPORT_DIR)

# 保存模型权重
torch.save(model.state_dict(), f'{EXPORT_DIR}/pytorch_model.bin')

# 复制分词器
!cp model/tokenizer.json {EXPORT_DIR}/
!cp model/tokenizer_config.json {EXPORT_DIR}/

print(f"✅ 模型已导出到: {EXPORT_DIR}")
!ls -la {EXPORT_DIR}

In [ ]:
# 压缩并下载 (Colab)
!zip -r minillm_0_2b.zip {EXPORT_DIR}

# 在 Colab 中下载
try:
    from google.colab import files
    files.download('minillm_0_2b.zip')
except ImportError:
    print("不在 Colab 环境中，请手动下载: minillm_0_2b.zip")

## 9️⃣ 训练技巧与调优

### A100 优化建议:

1. **批量大小**: A100 80GB 可支持 `batch_size=128` + `max_seq_len=2048`
2. **混合精度**: 使用 `bfloat16` 获得最佳性能
3. **Flash Attention**: 如果可用，启用 Flash Attention 2
4. **梯度累积**: 使用 `accumulation_steps` 模拟更大批量

### 分布式训练:

```bash
# 双卡 DDP 训练
torchrun --nproc_per_node 2 trainer/train_pretrain.py \
    --ddp \
    --hidden_size 768 \
    --num_hidden_layers 12 \
    ...
```

### 调试技巧:

```python
# 使用 --max_steps 进行快速验证
python trainer/train_pretrain.py --max_steps 100 ...
```

---

## 🎉 完成

恭喜！你已经成功在 A100 上训练了一个 0.2B 参数的 MiniLLM 模型。

**后续步骤建议:**
1. 使用更大的数据集进行完整预训练
2. 进行 SFT 微调以提升对话能力
3. 尝试 DPO/GRPO 等对齐方法
4. 使用 vLLM/SGLang 部署推理服务

**相关资源:**
- [MiniLLM GitHub](https://github.com/ai-clarify/mini-llm)
- [DeepSeek-V3 技术报告](https://arxiv.org/abs/2401.xxxxx)